In [ ]:
import keras
from keras import layers, callbacks
import tensorflow as tf
import pandas as pd
import numpy as np

from keras.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

VOCAB_SIZE = 15000
SEQUENCE_LENGTH = 100
EMBED_DIM = 8

DATA_PATH=r"../../data/transformed/amazon_reviews_5_partition_1.csv"

In [ ]:
df_reader = pd.read_csv(
    DATA_PATH,
    index_col=0,
    chunksize=10000
)

In [ ]:
df = df_reader.__next__()

In [ ]:
df = pd.read_csv(DATA_PATH, index_col=0)

In [ ]:
df.sample(4)

In [ ]:
df = df.dropna()
df["reviewText"] = df["reviewText"].astype("string")

In [ ]:
x = df["reviewText"]
y = df["overall"] -1

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
with open("../../checkpoints/vectorization_vocabulary.txt", "r") as file:
    vocab = file.read()
    vocab=vocab.split("\n")[:-1] #last line is an empty string

In [ ]:
vectorize_layer = layers.TextVectorization(
    VOCAB_SIZE,
    "lower_and_strip_punctuation",
    output_mode="int",
    output_sequence_length=SEQUENCE_LENGTH,
    vocabulary=vocab
)

In [ ]:
# Increase complexity

lstm_model = keras.Sequential(
    [
        layers.Input(shape=(1,), dtype="string"),
        vectorize_layer,
        layers.Embedding(VOCAB_SIZE, EMBED_DIM),
        layers.LSTM(124, dropout=0.5, return_sequences=True),
        layers.LSTM(16, dropout=0.5),
        layers.Dense(1)
    ]
)

In [ ]:
# Load from checkpoint

lstm_model.load_weights(r"../../checkpoints/lstm_model_24_24_250k_rows/lstm_model")

In [ ]:
lstm_model.compile(
    optimizer="adam", 
    loss="mse",
    metrics=["mae"]
)

In [ ]:
cbs = [
    callbacks.ModelCheckpoint(
        filepath="../../checkpoints/lstm_model_124_16_10k_rows",
        save_best_only=True,
        save_weights_only=True
    ),
    callbacks.EarlyStopping(
        patience=8
    )
]

In [ ]:
lstm_model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    validation_data=(x_test, y_test),
    callbacks=cbs
)

In [ ]:
lstm_model.predict([""])

In [ ]:
y_train_naive = np.full_like(y_train, dtype="float32", fill_value=2.1539134783695926)

In [ ]:
mean_absolute_error(y_train, y_train_naive).numpy()

In [ ]:
predicted = lstm_model.predict(x)

In [ ]:
stats = pd.DataFrame({
    "text":x,
    "actual":y,
    "predicted":predicted.flatten()
})

In [ ]:
stats.sample(10)